In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        continue
        #print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
# 랜덤 시드 고정
import random
import librosa
import matplotlib.pyplot as plt
from tqdm import tqdm
import time

seed = 42

random.seed(seed)
np.random.seed(seed)
os.environ["PYTHONHASHSEED"] = str(seed)

# 1. 학습 및 제출에 사용할 csv 파일 불러오기

In [3]:
# 학습할 음악들의 장르 정답 csv 파일 불러오기
train_info_csv = pd.read_csv('/kaggle/input/2023-ml-finalexam-pp5/train_labels.csv')

# 제출에 사용할 csv 파일 불러오기
submit = pd.read_csv('/kaggle/input/2023-ml-finalexam-pp5/submit_sample.csv')
test_info_csv = submit['id']

In [4]:
# 어떤 장르들을 분류해야 하는지 확인해보기
train_info_csv['genre'].unique()

array(['rock', 'country', 'metal', 'hiphop', 'pop', 'classical', 'disco',
       'reggae', 'blues', 'jazz'], dtype=object)

# 2. 데이터 불러오기 및 Handcrafted-Feature(MFCC) 추출

## Empty Module #1: extract_mfcc_feature()
첫 번째 Empty Module에서는 각 음악 파일에 대한 MFCC feature를 추출하는 함수를 구현합니다.  
단, 본 Empty Module에서는 **librosa, numpy 라이브러리의 함수를 통해 spectrum, spectrogram, power_spectrogram, melspectrogram, melspectrogram_db를 순차적으로 얻고,  
이를 입력으로 주어 MFCC feature를 얻는 것을 목표**로 합니다.

문제 Overview에 주어진 [안내 사항](https://www.kaggle.com/competitions/2023-ml-finalexam-pp5/overview)과 아래 붉은 글씨로 된 주의사항을 천천히 읽은 뒤 함수를 구현하시기 바랍니다.

<span style="color:red">    
    
- 만약 spectrum, spectrogram, power_spectrogram, melspectrogram, melspectrogram_db 구현 없이 time-series로부터 직접 MFCC feature를 추출하는 경우에는 베이스라인 성능을 달성하더라도 본 문제에 대한 점수를 인정받을 수 없습니다.  
    
- 즉, 아래 extract_mfcc_feature() 함수 내에 적혀 있는 변수 값들을 순차적으로 채워나가며 최종적인 MFCC feature를 추출하여 학습, 추론에 사용하시기 바랍니다.  
    
- 또한, 아래 함수에서 구현한 MFCC feature가 아닌 다른 feature로 베이스라인을 통과하는 경우 점수가 인정되지 않습니다.  
</span>


In [5]:
# Empty Module #1

def extract_mfcc_feature(file_path):
    
    feature = []
    
# 아래 빈 칸에 함수를 채워넣어 순차적으로 MFCC feature를 구현하세요.
# -------------------------------------------------------------------------------------------------
    y, sr = librosa.load(file_path, sr = 22050) #파일 불러오기
    
    spectrum = librosa.stft(y) #spectrogram으로 만들어주기

    spectrogram = np.abs(spectrum) #spectrogram을 절대값 씌어주기

    power_spectrogram = (spectrogram)**2 #제곱해주기
 
    melspectrogram = librosa.feature.melspectrogram(S=power_spectrogram) #melspectrogram만들기
    
    melspectrogram_db = librosa.power_to_db(melspectrogram) #db로 만들기

    mfcc = librosa.feature.mfcc(S=melspectrogram_db) #mfcc feature로 만들어주기
# -------------------------------------------------------------------------------------------------
     
    # 2차원으로 추출된 mfcc 값을 분류기에 넣어주기 위해 1차원 벡터로 변경해주어야 합니다.
    # 이는 아래 코드와 같이 시간축에 대한 평균을 내어 수행합니다.
    mfcc_feature = np.mean(mfcc, axis=1) 
    
    return mfcc_feature

In [6]:
def feature_loader(data_info, split=None, rootpath=None):
    split = split.upper()
    info_dict = {}
    
    if split=='TRAIN':
        train_path = os.path.join(rootpath, 'train')
        file_list = data_info['id']
        label_list = data_info['genre']
        
        for file, label in zip(tqdm(file_list), label_list):
            # ----------------------------------------------
            # 학습에 사용할 음성 중 손상된 wav 파일은 제외합니다.
            # 아래의 조건문은 수정하실 필요 없습니다.
            if file == 'train_412.wav':
                continue    
            # ----------------------------------------------
            file_dict = {}     
            file_dict['label'] = label
            file_path = os.path.join(train_path, file)
            features = extract_mfcc_feature(file_path)
            file_dict['features'] = features
            info_dict[file] = file_dict
            
        return info_dict
        
    elif split=='TEST':
        test_path = os.path.join(rootpath, 'test')
        file_list = data_info
        
        for file in tqdm(file_list):
            file_dict = {}
            file_path = os.path.join(test_path, file)
            features = extract_mfcc_feature(file_path)   
            file_dict['features'] = features
            info_dict[file] = file_dict
            
        return info_dict

In [7]:
rootpath = '/kaggle/input/2023-ml-finalexam-pp5'

train_data = feature_loader(train_info_csv, split='train', rootpath=rootpath)
test_data = feature_loader(test_info_csv, split='test', rootpath=rootpath) 

100%|██████████| 200/200 [00:28<00:00,  7.06it/s]


# 3. 학습, 테스트 데이터 구축 및 분류기 설계

In [8]:
x_train = []
y_train = []

for key in train_data.keys():
    x_train.append(train_data[key]['features'])
    y_train.append(train_data[key]['label'])
    
x_train = np.asarray(x_train)
y_train = np.asarray(y_train)

In [9]:
from sklearn.preprocessing import LabelEncoder

# 문자열로 된 음악의 장르들을 수치형으로 변경하기 위해 LabelEncoder를 사용합니다.
le = LabelEncoder()
y_train = le.fit_transform(y_train)

In [10]:
x_test = []

for key in test_data.keys():
    x_test.append(test_data[key]['features'])
    
x_test = np.asarray(x_test)

## Empty Module #2: 분류기 선언, 학습, 예측 수행
두 번째 Empty Module에서는 앞서 얻은 feature를 활용할 분류기를 설계합니다.  
베이스라인의 분류기는 RandomForestClassifier(random_state=seed)로, 분류기에 대한 별다른 하이퍼파라미터 튜닝을 진행하지 않은 성능입니다.

분류기를 선언하고 앞서 얻은 feature와 라벨로 fit, predict하는 과정을 구현하시면 됩니다.  

In [11]:
# Empty Module #2

# 아래 빈 칸에 함수를 채워넣어 학습 및 분류를 수행하세요.
# -------------------------------------------------------------------------------------------------
from sklearn.ensemble import RandomForestClassifier

model = RandomForestClassifier()
model.fit(x_train,y_train)
pred_rfc = model.predict(x_test)

In [12]:
# 'pred_rfc' 변수는 선언하신 예측값의 변수명에 맞게 수정해주세요.
submit['genre'] = le.inverse_transform(pred_rfc)
submit.to_csv('MFCC_feature_baseline.csv', index=False)